In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from scipy.stats import truncnorm, beta, norm, lognorm, triang
import os
from SALib.sample import saltelli
from SALib.analyze import sobol
import matplotlib.pyplot as plt

# ======================= 1. 参数设置 =======================
# --- 输入文件 ---
excel_path = r"G:\Remote sensing paper\RESAMPLE\new revise\Data.xlsx"

# --- 输出文件 ---
output_dir = r"G:\Remote sensing paper\RESAMPLE\new revise"
os.makedirs(output_dir, exist_ok=True)
output_iter_excel_base = os.path.join(output_dir, "iterations_results.xlsx") 
output_geotiff = os.path.join(output_dir, "geothermal_potential_mean.tif")
sobol_results_excel = os.path.join(output_dir, "sobol_sensitivity_results_high_sample.xlsx")
sobol_plot_path = os.path.join(output_dir, "sobol_sensitivity_plot_high_sample.png")

# --- 模拟参数 ---
n_iterations = 10000  # 蒙特卡洛模拟的迭代次数
batch_size = 1000     # 每批保存的迭代次数
sobol_n_samples = 4096 # Sobol分析的样本量

# --- 指标列名映射 ---
indicator_columns = {
    "A1": "A1_rock", "A2": "A2_fault", "A3": "A3_depth",
    "A4": "A4_temp", "A5": "A5_heatflow", "A6": "A6_mag",
    "A7": "A7_land"
}

# --- 权重参数 ---
triang_params = {
    "A1": (0.1489, 0.1530, 0.1532), "A2": (0.2124, 0.2140, 0.2182),
    "A3": (0.0879, 0.0917, 0.1022), "A4": (0.0541, 0.0582, 0.0603),
    "A5": (0.2853, 0.3212, 0.3344), "A6": (0.0946, 0.1012, 0.1155),
    "A7": (0.0607, 0.0636, 0.0695),
}
indicator_order = list(triang_params.keys()) 

# --- Sobol分析参数范围 ---
sobol_problem = {
    'num_vars': 14,
    'names': [
        'w_A1', 'w_A2', 'w_A3', 'w_A4', 'w_A5', 'w_A6', 'w_A7', # 权重参数
        'd_A1', 'd_A2', 'd_A3', 'd_A4', 'd_A5', 'd_A6', 'd_A7'  # 数据不确定性参数
    ],
    'bounds': [
        # 权重范围
        [0.1489, 0.1532], [0.2124, 0.2182], [0.0879, 0.1022], [0.0541, 0.0603],
        [0.2853, 0.3344], [0.0946, 0.1155], [0.0607, 0.0695],
        # 数据不确定性范围 (简化为[0,1]的因子)
        [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0],
        [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]
    ]
}

output_crs = "EPSG:4326"

# ======================= 2. 向量化采样函数定义  =======================
rock_params = {
    1: {"λ_mean": 2.80, "λ_std": 0.50, "k_mean": 3.00e-04, "k_std": 1.5e-04, "norm": 0.1},
    2: {"λ_mean": 2.70, "λ_std": 0.25, "k_mean": 1.65e-03, "k_std": 6.75e-04, "norm": 0.01},
    3: {"λ_mean": 1.90, "λ_std": 0.25, "k_mean": 1.00e-07, "k_std": 5.00e-08, "norm": 0.001},
    4: {"λ_mean": 3.00, "λ_std": 0.65, "k_mean": 3.00e-04, "k_std": 1.50e-04, "norm": 0.1},
    5: {"λ_mean": 2.20, "λ_std": 0.50, "k_mean": 5.01e-07, "k_std": 2.50e-07, "norm": 0.001},
    6: {"λ_mean": 3.15, "λ_std": 0.525,"k_mean": 1.50e-08, "k_std": 7.50e-09, "norm": 0.001},
    7: {"λ_mean": 0.94, "λ_std": 0.085,"k_mean": 1.00e-03, "k_std": 5.00e-04, "norm": 0.01},
}

def truncated_normal_vec(mean, std, low=0, high=np.inf, size=1):
    if isinstance(mean, (int, float)): mean = np.full(size, mean)
    if isinstance(std, (int, float)): std = np.full(size, std)
    
    # 防止std为0导致除法错误
    std[std <= 0] = 1e-9
    
    a, b = (low - mean) / std, (high - mean) / std
    return truncnorm.rvs(a, b, loc=mean, scale=std, size=size)

def sample_A1_vec(rock_col):
    results = np.zeros_like(rock_col, dtype=float)
    for rock_type, params in rock_params.items():
        mask = (rock_col == rock_type)
        count = np.sum(mask)
        if count == 0: continue
        
        λ = truncated_normal_vec(params["λ_mean"], params["λ_std"], size=count)
        k = truncated_normal_vec(params["k_mean"], params["k_std"], size=count)
        
        value = λ * k * 1e3
        results[mask] = value / params["norm"]
    return np.clip(results, 0, 1)

def sample_A2_vec(fault_col):
    size = len(fault_col)
    normalized_value = fault_col / 99.0
    rand_mask = np.random.rand(size) < 0.6
    results = beta.rvs(2, 5, size=size) * normalized_value
    results[rand_mask] = 0
    return np.clip(results, 0, 1)

def sample_A3_vec(depth_col):
    depth_m = (depth_col / 17.0) * 4000
    sampled = norm.rvs(depth_m, 100)
    return np.clip(1 - sampled / 4000, 0, 1)

def sample_A4_vec(temp_col):
    temp_c = (temp_col - 1) / 9.0 * 40
    sampled = norm.rvs(temp_c, 2)
    return np.clip(sampled / 40, 0, 1)

def sample_A5_vec(heatflow_col):
    size = len(heatflow_col)
    results = np.zeros(size, dtype=float)
    
    neg_mask = heatflow_col < 0
    pos_mask = ~neg_mask
    
    results[neg_mask] = np.random.choice([30, 40, 50, 60, 70, 80], size=np.sum(neg_mask)) / 100.0
    results[pos_mask] = norm.rvs(heatflow_col[pos_mask], 5) / 100.0
    return np.clip(results, 0, 1)

def sample_A6_vec(mag_col):
    normalized_value = (mag_col + 209) / 474.0
    lognorm_adjusted = lognorm.rvs(0.5, scale=normalized_value * 100)
    return np.clip(lognorm_adjusted / 100.0, 0, 1)

def sample_A7_vec(land_col):
    lulc_weights = {1: 0.3, 2: 0.2, 4: 0.4, 5: 0.5, 6: 0.1, 7: 0.1, 9: 0.2, 10: 0.9}
    base_values = land_col.map(lambda x: lulc_weights.get(x, 0.5)).to_numpy()
    sampled = np.random.normal(base_values, 0.1)
    return np.clip(sampled, 0, 1)

sampling_functions_vec = {
    "A1": sample_A1_vec, "A2": sample_A2_vec, "A3": sample_A3_vec, "A4": sample_A4_vec,
    "A5": sample_A5_vec, "A6": sample_A6_vec, "A7": sample_A7_vec,
}

# ======================= 3. 蒙特卡洛模拟 =======================

print("1. 读取Excel数据...")
try:
    df = pd.read_excel(excel_path)
    # 将原始数据列转换为NumPy数组以便快速访问
    raw_data = {ind: df[col].to_numpy() for ind, col in indicator_columns.items()}
except FileNotFoundError:
    print(f"错误：找不到文件 {excel_path}。请检查路径。")
    exit()

required_cols = ["X", "Y"] + list(indicator_columns.values())
for col in required_cols:
    if col not in df.columns:
        print(f"错误：Excel文件中缺少必需的列: '{col}'")
        exit()

print(f"   - 数据读取成功，共 {len(df)} 个点。")

# 初始化用于存储所有迭代结果的数组
results_matrix = np.zeros((len(df), n_iterations))

print(f"\n2. 开始执行 {n_iterations} 次蒙特卡洛模拟 (已优化)...")
for i in range(n_iterations):
    # --- a. 采样权重 ---
    sampled_weights_list = []
    for indicator in indicator_order:
        low, mode, high = triang_params[indicator]
        c = (mode - low) / (high - low) if (high - low) > 0 else 0.5
        sampled_weights_list.append(triang.rvs(c, loc=low, scale=high - low))
    
    weights = np.array(sampled_weights_list)
    weights /= np.sum(weights)

    # --- b. 向量化采样有利度得分 ---
    sampled_favorability = np.zeros((len(df), len(indicator_order)))
    for j, indicator in enumerate(indicator_order):
        sampler_func = sampling_functions_vec[indicator]
        if indicator == 'A7': 
             sampled_favorability[:, j] = sampler_func(df[indicator_columns[indicator]])
        else:
             sampled_favorability[:, j] = sampler_func(raw_data[indicator])
    
    # --- c. 计算本次迭代的加权综合得分 ---
    iteration_result = np.dot(sampled_favorability, weights)
    results_matrix[:, i] = iteration_result
    
    # 显示进度
    if (i + 1) % 500 == 0 or (i + 1) == n_iterations:
        print(f"   - 完成进度: {i+1}/{n_iterations} ({((i + 1) / n_iterations) * 100:.1f}%)")

print("   - 模拟完成。")

# ======================= 4. 结果保存 =======================

print("\n3. 分批保存迭代结果到Excel文件...")
n_batches = (n_iterations + batch_size - 1) // batch_size
for batch in range(n_batches):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, n_iterations)
    
    batch_iter_cols = [f"iter_{i+1}" for i in range(start_idx, end_idx)]
    batch_df = pd.DataFrame(results_matrix[:, start_idx:end_idx], columns=batch_iter_cols)
    
    batch_output_df = pd.concat([df[["X", "Y"]], batch_df], axis=1)
    
    batch_filename = output_iter_excel_base.replace('.xlsx', f'_batch_{batch+1}_of_{n_batches}.xlsx')
    batch_output_df.to_excel(batch_filename, index=False)
    print(f"   - 批次 {batch+1}/{n_batches} 已保存: {batch_filename}")
print("   - 所有批次保存完成。")


print("\n4. 计算平均值并生成GeoTIFF和汇总文件...")
mean_probability = results_matrix.mean(axis=1)
df["probability_mean"] = mean_probability

# 生成栅格文件
x_unique = np.sort(df["X"].unique())
y_unique = np.sort(df["Y"].unique())[::-1]
pixel_size_x = np.mean(np.diff(x_unique)) if len(x_unique) > 1 else 1.0
pixel_size_y = -np.mean(np.diff(y_unique)) if len(y_unique) > 1 else -1.0 
width = len(x_unique)
height = len(y_unique)
# from_origin(x_corner, y_corner, x_size, y_size)
transform = from_origin(x_unique[0] - pixel_size_x / 2, y_unique[0] - pixel_size_y / 2, pixel_size_x, -pixel_size_y)

grid = np.full((height, width), np.nan, dtype='float32')
df['row_idx'] = df['Y'].apply(lambda y: np.where(y_unique == y)[0][0])
df['col_idx'] = df['X'].apply(lambda x: np.where(x_unique == x)[0][0])
grid[df['row_idx'], df['col_idx']] = df['probability_mean']

meta = {'driver': 'GTiff', 'height': height, 'width': width, 'count': 1, 'dtype': 'float32',
        'crs': output_crs, 'transform': transform, 'nodata': np.nan}
with rasterio.open(output_geotiff, 'w', **meta) as dst:
    dst.write(grid, 1)
print(f"   - 平均概率栅格图已保存到: {output_geotiff}")

summary_df = df[["X", "Y", "probability_mean"]]
summary_filename = output_iter_excel_base.replace('.xlsx', '_summary.xlsx')
summary_df.to_excel(summary_filename, index=False)
print(f"   - 汇总文件已保存: {summary_filename}")

# ======================= 5. Sobol 敏感性分析 =======================
print("\n5. 开始执行 Sobol 敏感性分析...")

# --- a. 准备基准有利度数据 ---
base_favorability_from_mc = np.zeros((len(df), len(indicator_order)), dtype=np.float32)
for j, indicator in enumerate(indicator_order):
    sampler_func = sampling_functions_vec[indicator]
    if indicator == 'A7':
        base_favorability_from_mc[:, j] = sampler_func(df[indicator_columns[indicator]])
    else:
        base_favorability_from_mc[:, j] = sampler_func(raw_data[indicator])
print("   - Sobol分析的基准有利度数据已准备。")


# --- b. 定义Sobol模型函数  ---
def sobol_model_batch(params_batch):
    # params_batch 是一个 (batch_size, D) 的数组, D是参数个数 (14)
    weights_samples = params_batch[:, :7]
    data_uncertainty_samples = params_batch[:, 7:]
    
    # 归一化权重
    weights_samples /= np.sum(weights_samples, axis=1, keepdims=True)
    perturbed_favorability = (data_uncertainty_samples[:, np.newaxis, :] * base_favorability_from_mc[np.newaxis, :, :] +
                              (1 - data_uncertainty_samples[:, np.newaxis, :]) * (1 - base_favorability_from_mc[np.newaxis, :, :]))
    
    results_per_point = np.einsum('ij,ikj->ik', weights_samples, perturbed_favorability)
    return np.mean(results_per_point, axis=1)


# --- c. 生成参数样本 ---
param_values = saltelli.sample(sobol_problem, sobol_n_samples, calc_second_order=True)
print(f"   - Saltelli 样本已生成，总运行次数: {len(param_values)}")

# --- d. 分批次运行模型  ---
# 初始化输出数组
Y = np.zeros(param_values.shape[0])
# 定义批处理大小
sobol_batch_size = 1024  
num_batches = (len(param_values) + sobol_batch_size - 1) // sobol_batch_size

print("   - 开始分批次运行Sobol模型...")
for i in range(num_batches):
    start_idx = i * sobol_batch_size
    end_idx = min(start_idx + sobol_batch_size, len(param_values))
    
    # 获取当前批次的参数
    params_batch = param_values[start_idx:end_idx]
    
    # 对当前批次运行模型
    Y[start_idx:end_idx] = sobol_model_batch(params_batch)
    
    print(f"     - 已处理批次 {i+1}/{num_batches}")

print("   - Sobol 模型运行完成。")

# --- e. 执行分析并保存结果 ---
print("   - 开始进行 Sobol 分析...")
Si = sobol.analyze(sobol_problem, Y, print_to_console=False, calc_second_order=True)
print("   - Sobol 分析完成。")

# 保存结果到Excel
total_Si, first_Si, second_Si = Si.to_df()
with pd.ExcelWriter(sobol_results_excel) as writer:
    total_Si.to_excel(writer, sheet_name="Total_Order_Sensitivity")
    first_Si.to_excel(writer, sheet_name="First_Order_Sensitivity")
    second_Si.to_excel(writer, sheet_name="Second_Order_Sensitivity")
print(f"   - Sobol 敏感性分析结果已保存到: {sobol_results_excel}")

# 绘制并保存图表
fig, ax = plt.subplots(1, figsize=(12, 7))
total_Si.plot(kind='bar', y='ST', yerr='ST_conf', title='Sobol Total-Order Indices (ST)', ax=ax, capsize=4, legend=False)
ax.set_ylabel("Total-Order Index (ST)")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.savefig(sobol_plot_path, dpi=300)
print(f"   - Sobol 敏感性分析图表已保存到: {sobol_plot_path}")


print("\n✅ 所有任务处理完成！")

In [ ]:
import pandas as pd
import numpy as np
import os
import re 

# ======================= 参数设置 =======================
output_dir = r"G:\Remote sensing paper\RESAMPLE\new revise" 
batch_file_prefix = "iterations_results_batch_"
batch_file_suffix = "_of_" 
output_variance_excel = os.path.join(output_dir, "geothermal_potential_variance_summary.xlsx")

# ======================= 主程序 =======================

print("1. 搜寻并确认批次文件...")
all_files = [f for f in os.listdir(output_dir) if f.startswith(batch_file_prefix) and f.endswith(".xlsx")]

if not all_files:
    print(f"错误：在目录 '{output_dir}' 中未找到任何以 '{batch_file_prefix}' 开头的批次文件。")
    exit()

parsed_batches = []
max_batch_num = 0
for filename in all_files:
    match = re.search(r'_batch_(\d+)_of_(\d+)\.xlsx', filename)
    if match:
        current_batch = int(match.group(1))
        total_batches_in_name = int(match.group(2))
        parsed_batches.append((current_batch, total_batches_in_name, filename))
        max_batch_num = max(max_batch_num, total_batches_in_name)

if not parsed_batches:
    print("错误：未能从文件名中解析出批次信息。请检查文件名格式是否为 'iterations_results_batch_X_of_Y.xlsx'。")
    exit()


parsed_batches.sort(key=lambda x: x[0])
expected_batches = list(range(1, max_batch_num + 1))
found_batch_numbers = [pb[0] for pb in parsed_batches]

if set(expected_batches) != set(found_batch_numbers):
    print(f"警告：检测到批次文件缺失或不连续。预期的批次有 {max_batch_num} 个，但只找到了 {len(found_batch_numbers)} 个。")
    print(f"缺失批次: {set(expected_batches) - set(found_batch_numbers)}")

print(f"   - 找到 {len(parsed_batches)} 个批次文件，总计 {max_batch_num} 个批次。")

# 存储每个点所有迭代结果
# 键为 (X, Y) 坐标元组，值为包含10000个迭代结果的列表
point_iterations = {}

# 获取坐标点总数，用于进度显示
first_batch_path = os.path.join(output_dir, parsed_batches[0][2])
df_first = pd.read_excel(first_batch_path)
num_points = len(df_first)
print(f"   - 共 {num_points} 个空间点需要处理。")

print("\n2. 逐批次读取迭代结果并汇集...")
for batch_idx, (current_batch, total_batches_in_name, filename) in enumerate(parsed_batches):
    file_path = os.path.join(output_dir, filename)
    print(f"   - 正在读取批次 {current_batch}/{total_batches_in_name}：{filename}")

    try:
        df_batch = pd.read_excel(file_path)
        # 提取迭代结果列
        iteration_cols = [col for col in df_batch.columns if col.startswith('iter_')]

        # 遍历 DataFrame 的每一行 (即每个空间点)
        for idx, row in df_batch.iterrows():
            x, y = row['X'], row['Y']
            point_key = (x, y)

            # 获取当前批次的迭代结果
            current_iter_values = row[iteration_cols].tolist()

            # 将当前批次的结果添加到该点的总列表中
            if point_key not in point_iterations:
                point_iterations[point_key] = []
            point_iterations[point_key].extend(current_iter_values)

    except Exception as e:
        print(f"警告：读取文件 {filename} 时发生错误: {e}。跳过此文件。")
        continue

print("   - 所有批次文件读取完成。")

print("\n3. 计算每个点的方差...")
variance_results = []
processed_count = 0
for point_key, iter_values in point_iterations.items():
    if len(iter_values) > 1: # 至少需要两个值才能计算方差
        variance = np.var(iter_values)
    else:
        variance = np.nan # 如果只有一个或零个值，方差为 NaN

    variance_results.append({'X': point_key[0], 'Y': point_key[1], 'Variance': variance})
    processed_count += 1
    if processed_count % (num_points // 10) == 0 or processed_count == num_points:
        print(f"   - 已计算 {processed_count}/{num_points} 个点的方差。")

print("   - 方差计算完成。")

print("\n4. 保存方差汇总表...")
df_variance_summary = pd.DataFrame(variance_results)
df_variance_summary.to_excel(output_variance_excel, index=False)
print(f"   - 方差汇总表已保存到: {output_variance_excel}")

print("\n✅ 方差计算和保存任务完成！")